## Module: VegET functions
- Created on: April 2020
- VERSION: numpy, gdal, python 3
- AUTHOR: Stefanie Kagone, Gabe Parrish, Tony Butzer

In [1]:
import numpy as np

# === global variables ===
start_year = 2013
end_year = 2016
DOY = 123
i = 122

# array value of NDVI for testing
ndvi = np.array([[0.25, 0.27, 0.7, 0.28],[0.26, 0.7, 0.7, 0.25]])
pet = np.array([[1.22, 1.22, 1.22, 1.22],[1.22, 1.22, 1.22, 1.22]])
ppt = np.array([[0.0, 0.0, 12.0, 0.0],[0.0, 0.0, 3.0, 0.0]])
tavg = np.array([[0.6, 0.6, 0.7, 0.6],[10, -2, 8, 6]])
tmin = np.array([[-3.6, -3.6, -3.6, -3.6],[5, -4, 2, 5]])
tmax = np.array([[4.1, 4.1, 4.1, 4.1],[15, 2, 14, 7]])

interception = np.array([[14.5, 18, 18, 14],[14.5, 18, 18, 14]])
whc = np.array([[71, 43, 178, .2],[0.1, 0.7, 0.7, .2]])
field_capacity = np.array([[126, 78, 277, 277],[126, 78, 277, 277]])
saturation = np.array([[153, 43, 435, 435],[153, 43, 435, 435]])

yest_snwpck = np.array([[0.1, 0.7, 0.7, .2],[0.1, 0.7, 0.7, .2]])
yest_swf = np.array([[0.1, 0.7, 0.7, .2],[0.1, 0.7, 0.7, .2]])

print(saturation)

rf_low_thresh_temp = 0
rf_high_thresh_temp = 6
rf_value = 0.167

melt_factor = 0.06

varA = 1.25
varB = 0.2 

dc_coeff = 0.65  # drainage coefficient
rf_coeff = 1 - dc_coeff  # runoff coefficient
print(rf_coeff)

[[153  43 435 435]
 [153  43 435 435]]
0.35


### Function: inital soil water

1. precipitation bias corrected (if required)
2. effective precip
3. intercepted precip
4. snow melt factor
5. RAIN
6. SWE
7. snowmelt
8. snow pack
9. intial soil water  = previous day soil water balance + RAIN + snow_melt

In [2]:
def soil_water(i, ppt, interception, tmin, tmax, tavg, melt_factor, rf_high_thresh_temp, rf_low_thresh_temp, yest_swf=None,
                    yest_snwpck=None):


    # Effective PPT 
    effppt = ppt * (1 - (interception / 100.0))
    
    # Intercepted PPT
    intcep = ppt * (interception / 100.0)
    
    # Snow pack
    # melt rate
    # Usage: Creates a melt rate value based on the relationship between max and min air temperatureto determine the snow melt and from there the snow pack extent
#     tmin = ''
#     tmax = ''
    melt_rate = melt_factor * ((tmax*tmax)-(tmax-tmin))
    # initialize the snow melt factor array
    snow_melt_fac = np.zeros(ndvi.shape)
    # Where snomel it <= the high threshold temp set it equal to the melth factor r
    snow_melt_fac[tavg <= rf_high_thresh_temp] = melt_rate[tavg <= rf_high_thresh_temp] 
    # otherwise make it 0  -- Operation happens in-place like i += x --
    snow_melt_fac[tavg > rf_high_thresh_temp] = 0
    print(snow_melt_fac)
    
    print(snow_melt_fac)
    # np.where(tavg <= rf_high_tresh_temp, 0, melt_rate)
    
    if i == 0:
        # rain fraction
        # Usage: Creates a fraction value based on average temperature that determines if the incoming precipiation is falling as rain, slied, or snow.
        rain_frac[tavg <= rf_low_thresh_temp] = 0
        rain_frac[tavg >= rf_high_thresh_temp] = 1
        temp_diff_boolean = (tavg < rf_high_thresh_temp) | (tavg > rf_low_thresh_temp)
        rain_frac[temp_diff_boolean] = rf_value[temp_diff_boolean] * (tavg[temp_diff_boolean] - rf_high_thresh_temp)
        
        RAIN = rain_frac * effppt
        SWE = np.zeros(ndvi.shape)  # inital snowpack raster with only 0 values 
        snow_melt = SWE
        SNOW_pack = np.zeros(ndvi.shape)  # inital snowpack raster with only 0 values
        SWi = (whc * 0.5) + effppt + snow_melt
    else:
        # rain fraction - initialize the rain fraction array
        rain_frac = np.zeros(ndvi.shape)
        # Usage: Creates a fraction value based on average temperature that determines if the incoming precipiation is falling as rain, slied, or snow.
        rain_frac[tavg <= rf_low_thresh_temp] = 0
        rain_frac[tavg >= rf_high_thresh_temp] = 1
        temp_diff_boolean = (tavg < rf_high_thresh_temp) | (tavg > rf_low_thresh_temp)
        rain_frac[temp_diff_boolean] = rf_value * (tavg[temp_diff_boolean] - rf_high_thresh_temp)
        # rf = np.where(tavg <= rf_low_tresh_temp, 0, Con(ta >= rf_high_tresh_temp, 1, rf_value*(ta-rf_high_tresh_temp)))
        
        RAIN = rain_frac * effppt
        SWE = (1 - rain_frac) * effppt
        
        #snow_melt = np.where(melt_rate <= (SWE + snowpacklist[-1]), melt_rate, (SWE + snowpacklist[-1]))
        # initialize snow melt
        snow_melt = np.zeros(ndvi.shape) # todo - should this be yesterdasy snowlment and be put in function call??
        snow_melt_boolean = (melt_rate <= (SWE + yest_snwpck))
        snow_melt[snow_melt_boolean] = melt_rate[snow_melt_boolean]
        snow_melt_boolean2 = (melt_rate > (SWE + yest_snwpck))
        snow_melt[snow_melt_boolean2] = SWE[snow_melt_boolean2] + yest_snwpck[snow_melt_boolean2]
        
        # re-initialize SNWpk array
        SNWpk = np.zeros(ndvi.shape)
        SNWpk1 = yest_snwpck + SWE - snow_melt
        SNWpk[SNWpk1<0] = 0
        SNWpk[SNWpk1>=0] = SNWpk1[SNWpk1 >=0]
        # SNWpk = np.where(SNWpk1 < 0, 0, SNWpk1)
        SWi = yest_swf + RAIN + snow_melt
        print(SWi)
    
    
    return SWi

# When running the code in a separate function call you would hava a counter i and refer to yesterdays snowpach and swf as below
# init_soul_water(a, b, c, d, snopack_arr_lst[i-1])        
SWi = soil_water(i, ppt, interception, tmin, tmax, tavg, melt_factor, rf_high_thresh_temp, rf_low_thresh_temp, yest_swf, yest_snwpck)

[[ 0.5466  0.5466  0.5466  0.5466]
 [ 0.     -0.12    0.      2.82  ]]
[[ 0.5466  0.5466  0.5466  0.5466]
 [ 0.     -0.12    0.      2.82  ]]
[[ 0.2       1.2466   -7.462784  0.4     ]
 [ 0.2       0.58      3.86      0.4     ]]


### Function: Surface runoff

1. surface runoff
2. deep drainage

In [3]:
def surface_runoff(SWi, saturation, field_capacity, whc, rf_coeff, geodict=None):
    # Deep Drainage
    # DDrain occurs if SWi > WHC, amount of DDrain is SAT <> WHC with a maximum DDrain of SAT - WHC
    sat_fc = saturation - field_capacity
    rf1 = SWi - whc
    # rf = np.where(rf1 < 0, 0, rf1)
    rf = np.zeros(SWi.shape)
    rf1_boolean = (rf1 >= 0)
    rf[rf1_boolean] = rf1[rf1_boolean]
    
    #SRf = np.where(rf <= sat_fc, rf * rf_coeff, (rf - sat_fc) + rf_coeff * sat_fc)
    SRf = np.zeros(SWi.shape)
    SRf_boolean = (rf <= sat_fc)
    SRf[SRf_boolean] = rf[SRf_boolean] * rf_coeff
    SRf_boolean2 = (rf > sat_fc)
    print(SRf_boolean2)
    SRf_boolean2_test = ~SRf_boolean
    print(SRf_boolean2_test)
    SRf[SRf_boolean2] = (rf[SRf_boolean2] - sat_fc[SRf_boolean2]) + rf_coeff * sat_fc[SRf_boolean2]
    
    DDrain = rf - SRf
    
    # TODO - Use rasterio to get proj and geotransform and output the raster using the geodict
    if geodict is not None:
        # do stuff
        pass
    
    return DDrain, SRf

DDrain, SRf = surface_runoff(SWi, saturation, field_capacity, whc, rf_coeff)
print(DDrain, SRf)

[[False  True False False]
 [False  True False False]]
[[False  True False False]
 [False  True False False]]
[[  0.    -22.75    0.      0.13 ]
 [  0.065 -22.75    2.054   0.13 ]] [[ 0.    22.75   0.     0.07 ]
 [ 0.035 22.75   1.106  0.07 ]]


### Function: Veg_ET

1. Evapotranspiration
2. final soil water (after PPT,ET,.. is accounted for)

In [9]:
def veg_et(varA, varB, pet, ndvi, SWi):
    # etasw1A = np.zeros(ndvi.shape)
    etasw1 = np.zeros(ndvi.shape)
    etasw3 = np.zeros(ndvi.shape)
    etasw4 = np.zeros(ndvi.shape)
    etasw = np.zeros(ndvi.shape)
    SWf = np.zeros(ndvi.shape)
    
    etasw1A = (varA * ndvi + varB) * pet
    etasw1B = (varA * ndvi) * pet
    
    # etasw1 = ndvi > 0.4, etasw1A, etasw1B
    ndvi_boolean = (ndvi > 0.4)
    etasw1[ndvi_boolean] = etasw1A[ndvi_boolean]
    etasw1[~ndvi_boolean] = etasw1B[~ndvi_boolean]
    
    etasw2 = (SWi / (0.5 * whc)) * etasw1
    
    # etasw3 = SWi > (0.5 * WHC), etasw1, etasw2
    etasw3_boolean = (etasw3 > SWi)
    etasw3[etasw3_boolean] = etasw1[etasw3_boolean]
    etasw3[~etasw3_boolean] = etasw1[~etasw3_boolean]
    
    # etasw4 = etasw3 > SWi, SWi, etasw3
    etasw4_boolean = (etasw3 > SWi)
    etasw4[etasw4_boolean] = SWi[etasw4_boolean]
    etasw4[~etasw4_boolean] = etasw3[~etasw4_boolean]
    
    # etasw = etasw4 > WHC, WHC, etasw4
    etasw_boolean = (etasw4 > whc)
    etasw[etasw_boolean] = whc[etasw_boolean]
    etasw[~etasw_boolean] = etasw4[~etasw_boolean]
    
    SWf1 = SWi - etasw
    
    # SWf = SWi > WHC, (WHC - etasw), SWf1 < 0.0, 0.0, SWf1)
    SWf_boolean = (SWi > whc)
    SWf_boolean2 = (SWf1 < 0.0)
    
    SWf[SWf_boolean] = whc[SWf_boolean] - etasw[SWf_boolean]
    SWf[SWf_boolean2] = 0
    SWf[~SWf_boolean2] = SWf1[~SWf_boolean2]
    print(SWf)
    
    return etasw, SWf
    
etasw, SWf = veg_et(varA, varB, pet, ndvi, SWi)    
print(etasw, SWf)    

[[0.      0.83485 0.      0.2    ]
 [0.1     0.      3.16    0.2    ]]
[[ 0.2       0.41175  -7.462784  0.2     ]
 [ 0.1       0.58      0.7       0.2     ]] [[0.      0.83485 0.      0.2    ]
 [0.1     0.      3.16    0.2    ]]


In [37]:
# === Delaware River basin study area geo info for geotiff outputs ===

geodict = {transform = '| 0.00, 0.00,-77.02 |', 
           coord_sys = 'EPSG:4326', 
           h = int(3124), w = int(1938)}

# print('model concept test')
# import os
# import numpy as np
# import rasterio
# from matplotlib import pyplot as pl

# ds = rasterio.open(testfile)

# print('transform', ds.transform)
# print(ds.crs)
# print(type(ds.height), ds.width)
# band1 = ds.read(1)
# with rasterio.open('test_tif.tif', 'w', driver='GTiff', height=ds.height, width=ds.width, 
#                        count=1, dtype='float32', crs=ds.crs, transform=ds.transform) as wrast:
#     wrast.write(band1, indexes=1)
# #     write(supervised, indexes=1)
    
    
    


model concept test
transform | 0.00, 0.00,-77.02|
| 0.00,-0.00, 43.51|
| 0.00, 0.00, 1.00|
EPSG:4326
<class 'int'> 1938
